In [ ]:
!pip install tensorly

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import csv
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import tensorflow as tf        
import tensorly       
from tensorly.decomposition import non_negative_parafac
from tensorly.metrics import regression
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
userdata = pd.read_csv('../input/credit-card-approval-prediction/application_record.csv')
recorddata = pd.read_csv('../input/credit-card-approval-prediction/credit_record.csv')
print(recorddata)

In [ ]:
userdata['STATUS']= recorddata['STATUS']
userdata = userdata.loc[0:500,:] 
print(userdata.shape)
row= userdata.shape[0]
print(row)
a = [1,2,3]

#userdata['creditType'] = [ "Bad" if (x == '1' or x == '2' or x == '3' or x == '4' or x == '5') else "Good" for x in userdata.STATUS]
userdata['Status'] = [ 0 if (x == '0') else 1 for x in userdata.STATUS]

for i,j in userdata.iterrows():
    if j.STATUS == '1':
        j.Status = 1 
    elif j.STATUS == '2':
        j.Status = 2 
    elif j.STATUS == '3':
        j.Status = 3 
    elif j.STATUS == '4':
        j.Status = 4 
    elif j.STATUS == '5':
        j.Status = 5 
        
#randomly placing numbers in status
for i in range(0,200):
    x = random.randint(1,5)
    rowno = random.randint(0,500)
    userdata.iloc[rowno,19] = x
    
print(userdata['Status'].value_counts())

headers=list(userdata.columns)
print(headers)
a=list(userdata.loc[0])
print(a)


In [ ]:
userdata.to_csv('csv_to_submit.csv', index = False)
df=userdata.copy()



df['DAYS_EMPLOYED']=df['DAYS_EMPLOYED'].astype('category').cat.codes
df['FLAG_OWN_CAR']=df['FLAG_OWN_CAR'].astype('category').cat.codes
df['FLAG_OWN_REALTY']=df['FLAG_OWN_REALTY'].astype('category').cat.codes
df['NAME_EDUCATION_TYPE']=df['NAME_EDUCATION_TYPE'].astype('category').cat.codes
df['NAME_FAMILY_STATUS']=df['NAME_FAMILY_STATUS'].astype('category').cat.codes
df['NAME_HOUSING_TYPE']=df['NAME_HOUSING_TYPE'].astype('category').cat.codes
df['STATUS']=df['STATUS'].astype('category').cat.codes


df.corr(method ='kendall') 

In [ ]:
from sklearn.decomposition import PCA 
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(df)
principalDf = pd.DataFrame(principalComponents, columns = ['Economic_Indicator'])
principalDf['ID']=userdata['ID']
principalDf['Status']= userdata['Status']
principalDf['OCCUPATION_TYPE']= userdata['OCCUPATION_TYPE']
print(principalDf['Status'].value_counts())

In [ ]:
l = list(principalDf['OCCUPATION_TYPE'])
m = list(principalDf['ID'])
n = list(principalDf['Economic_Indicator'])

a = list(set(l))
b = list(set(m))
c = list(set(n))
print(a[2])
print(len(b))
print(len(set(n)))

Occupation = dict([(y,x+1) for x,y in enumerate(set(l))])
userlist = dict([(y,x+1) for x,y in enumerate(set(m))])
eco_in = dict([(int(y),x+1) for x,y in enumerate(set(n))])

#print(Occupation)
#print(userlist)

all_values = eco_in.values()
max_value = max(all_values) + 1

t = tf.ones([20,max_value,5002])
tensor = tensorly.tensor(t) 
#print(eco_in)

#print(tensor[0,0,0])
one=0
two=0
three=0
four=0
five=0
zero=0
indexlist = []
valuelist = []
I=0

for x in range(1,11):
    for y in range(1,92):
        for z in range(1,501):
                I+=1
                #print(I)
                tensor[x,y,z]=principalDf.loc[principalDf['ID'] == b[y], 'Status'].iloc[0]
                if tensor[x,y,z] == 1:
                    one+=1
                elif tensor[x,y,z] == 2:
                    two+=1
                elif tensor[x,y,z] == 3:
                    three+=1
                elif tensor[x,y,z] == 4:
                    four+=1
                elif tensor[x,y,z] == 5:
                    five+=1
                elif tensor[x,y,z] == 0:
                    zero+=1
                
                    
#t=tf.reshape(tensor,91636640 )                
#y, idx, count = tf.unique_with_counts(t)            
print(one)
print(two)
print(three)
print(four) 
print(five) 
print(zero)
print(I)

temp1d = tf.reshape(tensor,[9203680]) 

y,idx,count= tf.unique_with_counts(temp1d)
print(y.numpy())
print(count.numpy())
#print(indexlist)
#print(valuelist)

indices = tf.constant(indexlist)
updates = tf.constant(valuelist)

#print(tf.tensor_scatter_nd_update(tensor, indices, updates))


In [ ]:

R = 3
g = tf.keras.utils.normalize(tensor, axis=2, order=2)

#print(type(g))
#print(g.shape)
a = tensorly.tensor(g)
print(type(a))

for i in range(0,100):
    x = random.randint(0,11)
    y = random.randint(0,91)
    z = random.randint(0,500)
    a[x,y,z] = 0

tensor2=tf.convert_to_tensor(a)
print(tensor2)
print(type(a))
print(type(tensor2))
#print(tensor2.shape)
wei,factors = non_negative_parafac(tensor,rank = 4)
print(factors)
print(wei)

In [ ]:
print(len(factors))
for f in factors:
    print(f.shape)

In [ ]:
print(factors[0].shape)
print(factors[1].shape)
print(factors[2].shape)
print(factors[0])
print(factors[1][0])
print(factors[1][1])

In [ ]:
regenerated_tensor = tensorly.kruskal_to_tensor((wei,factors))
print(regenerated_tensor[0,0,0])
one=0
two=0
three=0
four=0
five=0
zero=0


In [ ]:
temp1d = tf.reshape(regenerated_tensor,[9203680])
tt=tf.dtypes.cast(temp1d, tf.int32)
y,idx,count= tf.unique_with_counts(tt)
print(y.numpy())
print(count.numpy())

In [ ]:
#Error measurement
import tensorflow
print("Root Mean Square error = "+str(tensorly.metrics.regression.RMSE(tensor, regenerated_tensor, axis=None)))
print("Mean squared error = " + str(tensorly.metrics.regression.MSE(tensor, regenerated_tensor, axis=None)))
m=tensorflow.keras.losses.MeanAbsoluteError()
m(tensor,regenerated_tensor).numpy

e= tf.keras.metrics.Accuracy()
e.update_state(tensor, regenerated_tensor)
pa=e.result().numpy()
print("Prediction Accuracy ="+ str(pa))